In [1]:
#https://github.com/usman-hussain/NHTSA-Complaints-Database-/blob/main/NHTSA%20Complaints%20Database.ipynb
# https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews
import json
import requests
from os.path import exists,join
from os import makedirs
import numpy as np
# import altair as alt
import pandas as pd

## Complaints

In [5]:
url0 = 'http://www.nhtsa.gov/webapi/api/Complaints/vehicle/modelyear/{}/make/{}/model/{}?format=json'
def get_complaints(make,model,year):
    folder = 'NHTSAcomplaints'
    if not exists(folder): makedirs(folder)
    filename = make + '_' + model + '_' + str(year) + '.json'
    path = join(folder,filename)
    if exists( path ):
       # print('Have previously downloaded this: getting from local file')
        with open(path) as f:
            results = json.load(f)
    else:
        #print('Haven\'t downloaded this one yet: getting from NHTSA and will save to local file')
        url = url0.format(year,make,model)
        results = json.loads(requests.get(url).text)
        with open(path,'w') as f:
            json.dump(results,f,indent=3)
    return results

In [6]:
carlist = [('Toyota','Tacoma'), ('Honda','Civic'),
        ('Toyota','Corolla'),('Toyota','Camry'),('Honda','CR-V'),
        ('Chevrolet', 'Silverado'),('Honda','Accord'), 
        ('Nissan', 'Altima'),  ('Honda', 'HR-V'), 
        ('Ford', 'Explorer'), ('Jeep', 'Wrangler'),  
        ('GMC', 'Sierra'),  ('Jeep', 'Grand Cherokee'), ('Toyota', 'RAV4'),  ('Ram', 'Pickup'),
          ('Ford', 'F-150')]
len(carlist)

16

In [7]:
from pandas.io.json import json_normalize
import pandas
data={ 'Crash':[], 'Fire':[], 
      'NumberOfInjured':[], 
      'NumberOfDeaths':[],
      'Component':[],
      'Component':[],
      'Summary':[],
      'ModelYear':[],
      'Make':[],
      'Model':[]
     }
df = pd.DataFrame()
for car in carlist:
#     print(car)

    make,model = car

    print()
# json_data = [] 
    for year in range(2000,2022):
        d = get_complaints(make,model,year)

    #     json_data.append( json_normalize(d['Results']) )
    #     print(json_data)
    # df_all = pandas.DataFrame.from_records( json_data )
        for result in d['Results']:

            data['Make'].append(result['Make'])
            data['Model'].append(result['Model'])
            data['Crash'].append(result['Crash'])
            data['Fire'].append(result['Fire'])
            data['NumberOfInjured'].append(result['NumberOfInjured'])
            data['NumberOfDeaths'].append(result['NumberOfDeaths'])
            data['Component'].append(result['Component'])
            data['Component'].append(result['Component'])
            data['Summary'].append(result['Summary'])           
            data['ModelYear'].append(result['ModelYear'])





In [8]:
df= pd.DataFrame.from_dict(data, orient='index')
df = df.T
df = df[df['Make'].notna()]

#change yes and no to 1 and 0
df['Crash'] = df['Crash'].replace({'Yes':1, 'No':0})
df['Fire'] = df['Fire'].replace({'Yes':1, 'No':0})

#remove special characters 
df['Component'] = df['Component'].str.replace('[^\w\s]', ' ')

C:\Users\Raza\AppData\Local\Temp\ipykernel_14680\469066589.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Component'] = df['Component'].str.replace('[^\w\s]', ' ')


In [10]:
df.to_csv('car_complaints_new.csv', index=False, encoding='utf-8')

### Plots

In [ ]:
df.Model.value_counts().plot(kind='bar', figsize = (12, 4))

In [2]:
import psycopg2
import os
import psycopg2.extras

In [11]:
df['ModelYear'] = df['ModelYear'].astype(int)
df['NumberOfInjured'] = df['NumberOfInjured'].astype(int)
df['NumberOfDeaths'] = df['NumberOfDeaths'].astype(int)

In [15]:
dtypes_replacement = {
    'object' : 'varchar',
    'int64' : 'int',
    'int32' : 'int',
    'float64' : 'float',
    'bool': 'BOOLEAN'
    
}
#generate appropriate datatypes for sql create command 
col_str = ", ".join("{} {}".format(n , d) for (n , d) in zip(df.columns, df.dtypes.replace(dtypes_replacement)))
col_str

'Crash int, Fire int, NumberOfInjured int, NumberOfDeaths int, Component varchar, Summary varchar, ModelYear int, Make varchar, Model varchar'

In [ ]:
# df['manufacturer'] = df['manufacturer'].str.replace('[^\w\s]', '')
# df['component'] = df['component'].str.replace('[^\w\s]', '')
# df['summary'] = df['summary'].str.replace('[^\w\s]', '')
# df['make'] = df['make'].str.replace('[^\w\s]', '')
# df['model'] = df['model'].str.replace('[^\w\s]', '')

# df['odinumber'] = df['odinumber'].astype(str).str.extract('(\d+)', expand=False)
# df['crash'] = df['crash'].astype(str).str.extract('(\d+)', expand=False)
# df['fire'] = df['fire'].astype(str).str.extract('(\d+)', expand=False)
# df['numberofinjured'] = df['numberofinjured'].astype(str).str.extract('(\d+)', expand=False)
# df['numberofdeaths'] = df['numberofdeaths'].astype(str).str.extract('(\d+)', expand=False)
# df['modelyear'] = df['modelyear'].astype(str).str.extract('(\d+)', expand=False)
# # df['airbag'] = df['airbag'].astype(str).str.extract('(\d+)', expand=False)

In [12]:
host_name = "localhost"
port_id = 5432
pwd = "test!123"
username = "postgres"
database = "nthsa"
conn = None
curr = None
# try:
conn = psycopg2.connect(
    host = host_name,
    dbname = database,
    user = username,
    password = pwd,
    port = port_id
)
curr = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)


In [16]:
#create table
#drop table if exists
curr.execute("drop table if exists complaints;")
#create table 
curr.execute("create table complaints \
            (Crash int, Fire int, NumberOfInjured int,\
            NumberOfDeaths int, Component varchar, Summary varchar, \
            ModelYear int, Make varchar, Model varchar)")


In [13]:
#read csv as object
#and insert into table
my_file = open('car_complaints_new.csv',encoding='utf8')
    
# except Exception as error:
#     print(error)
# finally:
#     if curr is not None:
#         curr.close()
#     if conn is not None:
#         conn.close()

sql_statement = """
COPY car_complaints FROM STDIN WITH
    CSV
    HEADER 
    DELIMITER AS ','
    NULL AS ' '
    encoding 'utf8'
"""

curr.copy_expert(sql=sql_statement, file=my_file)

In [16]:
select_all = """
select *  from car_complaints
"""
curr.execute(select_all)

results = curr.fetchall()

In [14]:
select_script = """
    select Make, count(*) as make_count
    from car_complaints
    group by make
    order by 1 desc;
"""

curr.execute(select_script)

for record in curr.fetchall():
    print(record['make'], record['make_count'])

TOYOTA 33496
NISSAN 12349
JEEP 27246
HONDA 33242
GMC 2028
FORD 32998
CHEVROLET 5685


#### insert dataframe to db table

In [21]:
select_script1 = """
    drop table if exists temp1;
    drop table if exists temp2;
    create  temporary table temp1
    as
        select model, modelyear,
        count(make) over (partition by modelyear) as model_count
        from car_complaints;
    create temporary table temp2
    as
        select *,
        row_number() over (partition by model, modelyear order by modelyear ) as row_numers
        from temp1;
    
    select model,modelyear, model_count
    from temp2 where row_numers = 1;    
"""

curr.execute(select_script1)

results = curr.fetchall()



# for record in curr.fetchall():
#     print(record['make'], record['modelyear'], record['model_count'], record['row_numers'])

In [23]:
count_script = """
    drop table if exists temp1;
    drop table if exists temp2;
    create temporary table temp1
    as
        select model, modelyear, 
            count(make) over (partition by modelyear) as model_count,
            sum(crash) over (partition by model, modelyear) as crash_count,
            sum(fire) over (partition by model, modelyear) as fire_count,
            sum(numberofinjured) over (partition by model, modelyear) as injured_count,
            sum(numberofdeaths) over (partition by model, modelyear) as death_count
        from car_complaints;
    create temporary table temp2
    as 
        select *,
            row_number() over (partition by model, modelyear order by modelyear) as row_numbers
        from temp1;
    select model, modelyear,model_count, crash_count, fire_count, injured_count, death_count
    from temp2 where row_numbers = 1 ;
    
"""

curr.execute(count_script)
count_results = curr.fetchall()


In [ ]:
headers = ['model', 'modelyear', 'model_count', 'crash_count', 
           'fire_count', 'injured_count', 'death_count']
counts_df = pd.DataFrame(count_results, columns=headers)

### Correlation

In [ ]:
#exclude model year
corr_df = counts_df[['model_count', 'crash_count', 'fire_count', 'injured_count', 'death_count']]
corr = corr_df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(corr_df)

In [ ]:
#between number of crashs and number of deaths
counts_df['death_count'].corr(counts_df['crash_count'])

In [ ]:
#between fires and death counts
counts_df['death_count'].corr(counts_df['fire_count'])